# Intro to Clustering

In [ ]:
%pylab inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import sklearn
import IPython
import platform

from sklearn import datasets
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Test set will be the 25% taken randomly
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.25, random_state=33)
    
# Standarise the features
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA

plt.figure()
#     plt.clf()
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X_train)
# X_2d = pca_2d.fit_transform(X_train[:,[0,1]])
# X_2d = X_train[:,[0,1]]
plt.scatter(X_2d[:,0], X_2d[:,1])
plt.title('Unmarked datapoints')
# plt.xlim(sl.min(), sl.max())
# plt.ylim(sw.min(), sw.max())
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
colormarkers = [ ['red','s'], ['greenyellow','o'], ['blue','x']]

Zs = None
#make a grid of numbers that span from min:max of sepal length and width
def make_bg_grid(clf, X_train):
    step = .01 
    margin = .1   
    sl_min, sl_max = X_train[:, 0].min()-margin, X_train[:, 0].max() + margin
    sw_min, sw_max = X_train[:, 1].min()-margin, X_train[:, 1].max() + margin
    #use numpy meshgrid to make a rectangular array from the limit values
    sl, sw  = np.meshgrid(
        np.arange(sl_min, sl_max, step), #arange creates an array that spans the specified range
        np.arange(sw_min, sw_max, step)
        )
    try:
        Zs = clf.predict( pca_2d.inverse_transform(np.c_[sl.ravel(), sw.ravel()])).reshape(sl.shape) # predict a Z value for each point in the grid
    except:
        print("Problem creating coloured backdrop")
        Zs = None
    return Zs, sl, sw

In [ ]:
from sklearn import cluster
clf_all = cluster.KMeans(init='k-means++', n_clusters=3, random_state=666)
clf_all.fit(X_train)

for clf in [clf_all]:
    Zs, sl, sw = make_bg_grid(clf, X_2d)
    centroids_s = clf.cluster_centers_
    try:
        tx_centroids = pca_2d.transform(centroids_s)
    except:
        tx_centroids = centroids_s
    
    #rjm49 - Now we've got the centroids, we can display the data points and the calculated regions, add the centroids as an extra scatter

#     plt.figure()
#     plt.clf()
    if Zs is not None:
        plt.imshow(Zs, interpolation='nearest', extent=(sl.min(), sl.max(), sw.min(), sw.max()), cmap= plt.cm.Pastel1, aspect='auto', origin='lower')
    for j in [0,1,2]:
#         px = X_train[:, 0][y_train == j]
#         py = X_train[:, 1][y_train == j]
        px = X_2d[:, 0][y_train == j]
        py = X_2d[:, 1][y_train == j]

        plt.scatter(px, py, c=colormarkers[j][0], marker= colormarkers[j][1])
    plt.scatter(tx_centroids[:, 0], tx_centroids[:, 1],marker='*',linewidths=3, color='black', zorder=10)
    plt.title('K-means clustering on the Iris dataset using all dimensions PCA\'d\nCentroids are marked with stars')
    plt.xlim(sl.min(), sl.max())
    plt.ylim(sw.min(), sw.max())
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()

What happens if we grow the number of clusters?

In [ ]:
for n_clus in [2,3,5,10]:
    clf = cluster.KMeans(init='k-means++', n_clusters=n_clus, random_state=33)
    clf.fit(X_train)

#     Zs = clf.predict( pca_2d.inverse_transform(np.c_[sl.ravel(), sw.ravel()])).reshape(sl.shape) # predict a Z value for each point in the grid
    Zs, sl, sw = make_bg_grid(clf, X_2d)
#     plt.figure()
    plt.imshow(Zs, interpolation='nearest', extent=(sl.min(), sl.max(), sw.min(), sw.max()), cmap= plt.cm.Pastel1, aspect='auto', origin='lower')
    for j in [0,1,2]:
        px = X_2d[:, 0][y_train == j]
        py = X_2d[:, 1][y_train == j]
        plt.scatter(px, py, c=colormarkers[j][0], marker= colormarkers[j][1])
#         plt.scatter(px,py, c=plt.cm.Dark2.colors[j])
    centroids_s = clf.cluster_centers_
    centroids_s = pca_2d.transform(centroids_s)
    plt.scatter(centroids_s[:, 0], centroids_s[:, 1],marker='*',linewidths=3, color='black', zorder=10)
    plt.title('K-means clustering on the Iris dataset\nAll dims, PCA\'d, k={}\nCentroids are marked with stars'.format(n_clus))
    plt.xlim(sl.min(), sl.max())
    plt.ylim(sw.min(), sw.max())
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()

## Estimating K

In [ ]:
# For higher dim datasets, may be preferable to do PCA first then scree plot
# pca = PCA(n_components=4)
# X_scree = pca.fit_transform(X_train)
X_scree = X_train

ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = cluster.KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(X_scree)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('Number of clusters, k')
plt.ylabel('Model inertia')
plt.xticks(ks)
plt.show()

## The Big Bad Clustering Chart
Many types of clustering are available, all with strengths and weaknesses.  For example Affinity Propagation does not require you to specify a number of clusters (but has other parameters instead) and Spectral Clustering can pick out concave and interlocking shapes (whereas K-means just finds blobs).  However they are more complicated to train and use (see the training times given below).

This chart gives some of the main ones and shows them in use on several synthetic datasets with various characteristics.

Much more info about clustering can be found at:
https://scikit-learn.org/stable/modules/clustering.html#clustering

In [ ]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
# Experimental OPTICS class
#     optics = cluster.OPTICS(min_samples=params['min_samples'],
#                             xi=params['xi'],
#                             min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(random_state=666,
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ag.C Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
#         ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()

## Summary

- We used k-means clustering to group our flower data into sub-categories (without using the labels).
- We got to grips with some more complicated graph plotting code
- We varied the number of clusters to see how we can force k-means to find sub-categories
- We used a scree plot of model inertia to estimate a good value for k
- We checked out the Big Bad Clustering Chart for alternative clustering techniques